# Function Call Intro

## 01. Librerías

In [ ]:
import os
import json
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI

## 02. Variables de Entorno

In [ ]:
# load the environment variables
load_dotenv()

# get the API key from the environment variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GPT4O_MODEL = os.getenv("OPENAI_GPT4O_MODEL")
GPT35_MODEL = os.getenv("OPENAI_GPT35_MODEL")

# print the API key and model
print(f"API Key: {OPENAI_API_KEY[:4]}...{OPENAI_API_KEY[-4:]}")
print(f"Model: {GPT4O_MODEL}")
print(f"Model: {GPT35_MODEL}")

## 03. Cliente OpenAI

In [ ]:
# create the OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

## 04. Funcion Chat Completion Request

In [ ]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT4O_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

## 05. Ejemplos de Matematicas

In [ ]:
# two random numbers
a = 122123
b = 256321

In [ ]:
query = f"Cuanto es {a} + {b}?"

response = chat_completion_request([{"role": "user", "content": query}])
content = response.choices[0].message.content

print(f"Query: {query}")
print(f"Response: {content}")
print(f"Real answer: {a+b}")

In [ ]:
query = f"Cuanto es {a} * {b}?"

response = chat_completion_request([{"role": "user", "content": query}])
content = response.choices[0].message.content

print(f"Query: {query}")
print(f"Response: {content}")
print(f"Real answer: {a*b}")

## 06. Definicion de Tools

In [ ]:
sumar_scheme = {
    "type": "function",
    "function": {
        "name": "sumar",
        "description": "Sumar dos números",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "Primer número",
                },
                "b": {
                    "type": "integer",
                    "description": "Segundo número",
                },
            },
            "required": ["a", "b"],
        },
    }
}

In [ ]:

multiplicar_scheme = {
    "type": "function",
    "function": {
        "name": "multiplicar",
        "description": "Multiplicar dos números",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "Primer número",
                },
                "b": {
                    "type": "integer",
                    "description": "Segundo número",
                },
            },
            "required": ["a", "b"],
        },
    }
}

## 07. Ejemplos de Function Call

In [ ]:
query = f"Cuanto es {a} + {b}?"

response = chat_completion_request([{"role": "user", "content": query}], tools=[sumar_scheme, multiplicar_scheme])
content = response.choices[0].message.content
tool_calls = response.choices[0].message.tool_calls

print(f"Query: {query}")
print(f"Response: {content}")
if tool_calls:
    print("Tool Calls:")
    for tool in tool_calls:
        print(f" - {tool.function}")

## 08. Ejecucion de Function Call

In [ ]:
def sumar(a: int, b: int) -> int:
    return a + b

def multiplicar(a: int, b: int) -> int:
    return a * b

def execute_tool(tool_name, parameters):
    parameters = json.loads(parameters)
    if tool_name == "sumar":
        result = sumar(parameters["a"], parameters["b"])
    elif tool_name == "multiplicar":
        result = multiplicar(parameters["a"], parameters["b"])
    return result

In [ ]:
query = f"Cuanto es {a} * {b}?"

response = chat_completion_request([{"role": "user", "content": query}], tools=[sumar_scheme, multiplicar_scheme])
content = response.choices[0].message.content
tool_calls = response.choices[0].message.tool_calls

print(f"Query: {query}")
print(f"Response: {content}")
if tool_calls:
    print("Tool Calls:")
    for tool in tool_calls:
        tool_result = execute_tool(tool.function.name, tool.function.arguments)
        print(f" - {tool.function} = {tool_result}")